# Language modeling: Character level
___
#### Description: 
In this notebook I use a recurrent neural network (RNN) to generate new Pokemon names. The model is trained using a many-to-one RNN where the input is a sequence of characters and the output is the character that follows the sequence. 
___
#### Dataset: 
The dataset contains all 807 Pokemon names.
___
#### Reference: 

This notebook was completed using the following resources as a guide:

https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/

https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py

In [1]:
# Get data
import pandas as pd
df = pd.read_csv('Datasets/pokemon.csv')
df.head(3)

,index,name
0,1,Bulbasaur
1,2,Ivysaur
2,3,Venusaur


In [2]:
# Extract names from dataframe
import numpy as np

# List containing all names in lowercase with a newline appended at end
names = [name.lower()+'\n' for name in df['name']]

In [3]:
# The newline indicates when a name ends
names[0:10]

['bulbasaur\n',
 'ivysaur\n',
 'venusaur\n',
 'charmander\n',
 'charmeleon\n',
 'charizard\n',
 'squirtle\n',
 'wartortle\n',
 'blastoise\n',
 'caterpie\n']

In [4]:
# Extract all unique characters from names
all_names = ''.join(names)
chars = sorted(set(all_names))
print(chars)

['\n', ' ', '-', '.', '2', ':', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é', '’', '♀', '♂']


In [5]:
# Create dictionaries that maps characters to indices and vice versa
char_indices = dict((c, i+1) for i, c in enumerate(chars))
indices_char = dict((i+1, c) for i, c in enumerate(chars))

In [6]:
# 0th index reserved for padding
print(indices_char)

{1: '\n', 2: ' ', 3: '-', 4: '.', 5: '2', 6: ':', 7: 'a', 8: 'b', 9: 'c', 10: 'd', 11: 'e', 12: 'f', 13: 'g', 14: 'h', 15: 'i', 16: 'j', 17: 'k', 18: 'l', 19: 'm', 20: 'n', 21: 'o', 22: 'p', 23: 'q', 24: 'r', 25: 's', 26: 't', 27: 'u', 28: 'v', 29: 'w', 30: 'x', 31: 'y', 32: 'z', 33: 'é', 34: '’', 35: '♀', 36: '♂'}


In [7]:
# Convert each name to a list of integers using char_indices
names_ind = []
for name in names:
    name_ind = []
    for char in name:
        name_ind.append(char_indices[char])
    names_ind.append(name_ind)       

In [9]:
# First 10 Pokemon names as a list of integers
names_ind[:10]

[[8, 27, 18, 8, 7, 25, 7, 27, 24, 1],
 [15, 28, 31, 25, 7, 27, 24, 1],
 [28, 11, 20, 27, 25, 7, 27, 24, 1],
 [9, 14, 7, 24, 19, 7, 20, 10, 11, 24, 1],
 [9, 14, 7, 24, 19, 11, 18, 11, 21, 20, 1],
 [9, 14, 7, 24, 15, 32, 7, 24, 10, 1],
 [25, 23, 27, 15, 24, 26, 18, 11, 1],
 [29, 7, 24, 26, 21, 24, 26, 18, 11, 1],
 [8, 18, 7, 25, 26, 21, 15, 25, 11, 1],
 [9, 7, 26, 11, 24, 22, 15, 11, 1]]

In [10]:
# Create sequences
sequences = []
for name_ind in names_ind:
    for i in range(1, len(name_ind)):
        sequence = name_ind[:i+1]
        sequences.append(sequence)

In [12]:
# The first 20 sequences
sequences[:20]

[[8, 27],
 [8, 27, 18],
 [8, 27, 18, 8],
 [8, 27, 18, 8, 7],
 [8, 27, 18, 8, 7, 25],
 [8, 27, 18, 8, 7, 25, 7],
 [8, 27, 18, 8, 7, 25, 7, 27],
 [8, 27, 18, 8, 7, 25, 7, 27, 24],
 [8, 27, 18, 8, 7, 25, 7, 27, 24, 1],
 [15, 28],
 [15, 28, 31],
 [15, 28, 31, 25],
 [15, 28, 31, 25, 7],
 [15, 28, 31, 25, 7, 27],
 [15, 28, 31, 25, 7, 27, 24],
 [15, 28, 31, 25, 7, 27, 24, 1],
 [28, 11],
 [28, 11, 20],
 [28, 11, 20, 27],
 [28, 11, 20, 27, 25]]

In [13]:
# Set the max length to be the length of the longest pokemon name (including \n)
maxlen = len(max(names_ind, key=len))
print('Max length:', maxlen)

Max length: 13


In [15]:
# Pad every sequence to be of the same length
from keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences, maxlen=maxlen, padding='pre')

In [16]:
# These sequences include the input and output and are required to have the same length
print(sequences[:20])

[[ 0  0  0  0  0  0  0  0  0  0  0  8 27]
 [ 0  0  0  0  0  0  0  0  0  0  8 27 18]
 [ 0  0  0  0  0  0  0  0  0  8 27 18  8]
 [ 0  0  0  0  0  0  0  0  8 27 18  8  7]
 [ 0  0  0  0  0  0  0  8 27 18  8  7 25]
 [ 0  0  0  0  0  0  8 27 18  8  7 25  7]
 [ 0  0  0  0  0  8 27 18  8  7 25  7 27]
 [ 0  0  0  0  8 27 18  8  7 25  7 27 24]
 [ 0  0  0  8 27 18  8  7 25  7 27 24  1]
 [ 0  0  0  0  0  0  0  0  0  0  0 15 28]
 [ 0  0  0  0  0  0  0  0  0  0 15 28 31]
 [ 0  0  0  0  0  0  0  0  0 15 28 31 25]
 [ 0  0  0  0  0  0  0  0 15 28 31 25  7]
 [ 0  0  0  0  0  0  0 15 28 31 25  7 27]
 [ 0  0  0  0  0  0 15 28 31 25  7 27 24]
 [ 0  0  0  0  0 15 28 31 25  7 27 24  1]
 [ 0  0  0  0  0  0  0  0  0  0  0 28 11]
 [ 0  0  0  0  0  0  0  0  0  0 28 11 20]
 [ 0  0  0  0  0  0  0  0  0 28 11 20 27]
 [ 0  0  0  0  0  0  0  0 28 11 20 27 25]]


In [17]:
# Number of unique characters + 1
vocab_size = len(chars) + 1 # including 0th index
print('Vocab size:', vocab_size)

Vocab size: 37


In [18]:
# Split sequences into inputs and outputs
from keras.utils import to_categorical

X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size) # convert y to one-hot vector

print('X shape:', X.shape)
print('y shape:', y.shape)

X shape: (6032, 12)
y shape: (6032, 37)


In [19]:
# Each output is represented as one-hot vectors of length vocab_size
print(y[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [20]:
# Build a RNN
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(vocab_size, 10, input_length=X.shape[1]),
    LSTM(50, dropout=0.2),
    Dense(vocab_size, activation='softmax')]
)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 12, 10)            370       
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 37)                1887      
Total params: 14,457
Trainable params: 14,457
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
# Compile and fit the model to X and y
from keras.callbacks import ModelCheckpoint

# Checkpoint weights after every epoch (optional)
checkpoint = ModelCheckpoint('weights-{epoch:02d}-{acc:.2f}.hdf5')

# Compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit
model.fit(X, y, epochs=500, callbacks=[checkpoint])

"""
Note:
Dropout is used so generated names vary from original names (less overfitting). This will 
result in a lower training accuracy which isn't a problem in our case of generating names.
Also, an embedding layer isn't required since characters are more or less independent from
one another.
"""

Epoch 1/500
6032/6032 [==============================] - 7s 1ms/step - loss: 2.7415 - acc: 0.2066
Epoch 2/500
6032/6032 [==============================] - 6s 936us/step - loss: 2.6711 - acc: 0.2245
Epoch 3/500
6032/6032 [==============================] - 6s 938us/step - loss: 2.6153 - acc: 0.2197
Epoch 4/500
6032/6032 [==============================] - 6s 936us/step - loss: 2.5819 - acc: 0.2200
Epoch 5/500
6032/6032 [==============================] - 6s 929us/step - loss: 2.5624 - acc: 0.2233
Epoch 6/500
6032/6032 [==============================] - 6s 926us/step - loss: 2.5496 - acc: 0.2258
Epoch 7/500
6032/6032 [==============================] - 6s 929us/step - loss: 2.5347 - acc: 0.2316
Epoch 8/500
6032/6032 [==============================] - 6s 933us/step - loss: 2.5203 - acc: 0.2339
Epoch 9/500
6032/6032 [==============================] - 6s 930us/step - loss: 2.5109 - acc: 0.2341
Epoch 10/500
6032/6032 [==============================] - 6s 931us/step - loss: 2.5019 - acc: 0.2389
E

6032/6032 [==============================] - 5s 903us/step - loss: 2.0973 - acc: 0.3377
Epoch 83/500
6032/6032 [==============================] - 6s 912us/step - loss: 2.1027 - acc: 0.3380
Epoch 84/500
6032/6032 [==============================] - 5s 910us/step - loss: 2.0849 - acc: 0.3432
Epoch 85/500
6032/6032 [==============================] - 5s 897us/step - loss: 2.0872 - acc: 0.3279
Epoch 86/500
6032/6032 [==============================] - 6s 927us/step - loss: 2.0886 - acc: 0.3370
Epoch 87/500
6032/6032 [==============================] - 6s 913us/step - loss: 2.0772 - acc: 0.3379
Epoch 88/500
6032/6032 [==============================] - 6s 917us/step - loss: 2.0716 - acc: 0.3483
Epoch 89/500
6032/6032 [==============================] - 5s 900us/step - loss: 2.0757 - acc: 0.3374
Epoch 90/500
6032/6032 [==============================] - 6s 922us/step - loss: 2.0573 - acc: 0.3468
Epoch 91/500
6032/6032 [==============================] - 6s 913us/step - loss: 2.0583 - acc: 0.3473
Epo

6032/6032 [==============================] - 6s 912us/step - loss: 1.8265 - acc: 0.4153
Epoch 163/500
6032/6032 [==============================] - 6s 913us/step - loss: 1.8243 - acc: 0.4158
Epoch 164/500
6032/6032 [==============================] - 6s 912us/step - loss: 1.8176 - acc: 0.4179
Epoch 165/500
6032/6032 [==============================] - 6s 920us/step - loss: 1.8106 - acc: 0.4171
Epoch 166/500
6032/6032 [==============================] - 6s 920us/step - loss: 1.8190 - acc: 0.4183
Epoch 167/500
6032/6032 [==============================] - 6s 918us/step - loss: 1.8080 - acc: 0.4203
Epoch 168/500
6032/6032 [==============================] - 6s 920us/step - loss: 1.8145 - acc: 0.4131
Epoch 169/500
6032/6032 [==============================] - 6s 916us/step - loss: 1.8025 - acc: 0.4231
Epoch 170/500
6032/6032 [==============================] - 6s 915us/step - loss: 1.8081 - acc: 0.4242
Epoch 171/500
6032/6032 [==============================] - 5s 909us/step - loss: 1.7998 - acc: 0

6032/6032 [==============================] - 5s 897us/step - loss: 1.6739 - acc: 0.4620
Epoch 243/500
6032/6032 [==============================] - 5s 902us/step - loss: 1.6728 - acc: 0.4622
Epoch 244/500
6032/6032 [==============================] - 5s 903us/step - loss: 1.6707 - acc: 0.4630
Epoch 245/500
6032/6032 [==============================] - 5s 906us/step - loss: 1.6580 - acc: 0.4720
Epoch 246/500
6032/6032 [==============================] - 5s 901us/step - loss: 1.6823 - acc: 0.4622
Epoch 247/500
6032/6032 [==============================] - 5s 903us/step - loss: 1.6579 - acc: 0.4620
Epoch 248/500
6032/6032 [==============================] - 5s 904us/step - loss: 1.6684 - acc: 0.4589
Epoch 249/500
6032/6032 [==============================] - 5s 896us/step - loss: 1.6676 - acc: 0.4629
Epoch 250/500
6032/6032 [==============================] - 5s 894us/step - loss: 1.6694 - acc: 0.4622
Epoch 251/500
6032/6032 [==============================] - 5s 902us/step - loss: 1.6485 - acc: 0

6032/6032 [==============================] - 6s 918us/step - loss: 1.5697 - acc: 0.4843
Epoch 323/500
6032/6032 [==============================] - 5s 910us/step - loss: 1.5730 - acc: 0.4849
Epoch 324/500
6032/6032 [==============================] - 6s 913us/step - loss: 1.5805 - acc: 0.4869
Epoch 325/500
6032/6032 [==============================] - 6s 916us/step - loss: 1.5787 - acc: 0.4833
Epoch 326/500
6032/6032 [==============================] - 5s 911us/step - loss: 1.5624 - acc: 0.4917
Epoch 327/500
6032/6032 [==============================] - 6s 916us/step - loss: 1.5731 - acc: 0.4799
Epoch 328/500
6032/6032 [==============================] - 6s 917us/step - loss: 1.5596 - acc: 0.4950
Epoch 329/500
6032/6032 [==============================] - 6s 920us/step - loss: 1.5684 - acc: 0.4867
Epoch 330/500
6032/6032 [==============================] - 6s 913us/step - loss: 1.5746 - acc: 0.4864
Epoch 331/500
6032/6032 [==============================] - 6s 919us/step - loss: 1.5695 - acc: 0

6032/6032 [==============================] - 6s 931us/step - loss: 1.5061 - acc: 0.5028
Epoch 403/500
6032/6032 [==============================] - 6s 934us/step - loss: 1.5121 - acc: 0.5050
Epoch 404/500
6032/6032 [==============================] - 6s 930us/step - loss: 1.5105 - acc: 0.5036
Epoch 405/500
6032/6032 [==============================] - 6s 934us/step - loss: 1.5146 - acc: 0.5050
Epoch 406/500
6032/6032 [==============================] - 6s 933us/step - loss: 1.5125 - acc: 0.5000
Epoch 407/500
6032/6032 [==============================] - 6s 934us/step - loss: 1.5052 - acc: 0.5101
Epoch 408/500
6032/6032 [==============================] - 6s 931us/step - loss: 1.5071 - acc: 0.5083
Epoch 409/500
6032/6032 [==============================] - 6s 936us/step - loss: 1.4994 - acc: 0.5093
Epoch 410/500
6032/6032 [==============================] - 6s 936us/step - loss: 1.4934 - acc: 0.5119
Epoch 411/500
6032/6032 [==============================] - 6s 936us/step - loss: 1.5091 - acc: 0

6032/6032 [==============================] - 6s 923us/step - loss: 1.4709 - acc: 0.5159
Epoch 483/500
6032/6032 [==============================] - 6s 921us/step - loss: 1.4550 - acc: 0.5249
Epoch 484/500
6032/6032 [==============================] - 6s 920us/step - loss: 1.4579 - acc: 0.5252
Epoch 485/500
6032/6032 [==============================] - 6s 918us/step - loss: 1.4623 - acc: 0.5172
Epoch 486/500
6032/6032 [==============================] - 6s 915us/step - loss: 1.4572 - acc: 0.5234
Epoch 487/500
6032/6032 [==============================] - 6s 920us/step - loss: 1.4482 - acc: 0.5206
Epoch 488/500
6032/6032 [==============================] - 6s 919us/step - loss: 1.4488 - acc: 0.5211
Epoch 489/500
6032/6032 [==============================] - 6s 915us/step - loss: 1.4598 - acc: 0.5229
Epoch 490/500
6032/6032 [==============================] - 6s 920us/step - loss: 1.4701 - acc: 0.5161
Epoch 491/500
6032/6032 [==============================] - 6s 916us/step - loss: 1.4622 - acc: 0

"\nNote:\nDropout is used so generated names vary from original names (less overfitting). This will \nresult in a lower training accuracy which isn't a problem in our case of generating names.\nAlso, an embedding layer isn't required since characters are more or less independent from\none another.\n"

In [36]:
# Generate pokemon names
import sys

# Prompt user input
text = input('Insert text: ')

print('======== Generated name ========')
sys.stdout.write(text)

# Convert inputted text to a list of character indices
X_user = [char_indices[char] for char in text]

# Pad X_user to have the same length as our training examples
X_user = pad_sequences([X_user], maxlen=X.shape[1], padding='pre')

# Repeatedly generate a new character and update X_user to include it
for i in range(maxlen):
    # Prediction on X_user
    y_pred = model.predict(X_user)

    # Sample from y_pred and convert to character
    sample = np.random.choice(vocab_size, 1, p = y_pred.ravel())
    next_char = indices_char[sample[0]]
    
    # Break out of loop if the predicted character is a newline
    if next_char == '\n':
        break
    
    # Print the sampled character and update X_user to include it
    sys.stdout.write(next_char)
    X_user = (np.append(X_user[0][1:], (sample[0]))).reshape(1,-1)


Insert text: be
======== Generated name ========
benamit

In [33]:
# Save the model and the summary for it
model.save('pokemon_name_generator.h5')

with open('pokemon_name_generator.txt', 'w+') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))